In [1]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI


In [2]:
load_dotenv(override=True)
openai = OpenAI()
gpt_model = 'gpt-4o-mini'

In [3]:
system_prompt = "You are a helpful assistant for an Airline called FlightAI. "
system_prompt += "Give short, courteous answers, no more than 1 sentence. "
system_prompt += "Always be accurate. If you don't know the answer, say so."

In [4]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history
    messages += [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return response.choices[0].message.content

In [5]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [6]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [7]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
tools = [{"type": "function", "function": price_function}]

In [9]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history
    messages += [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        tool_calls_message = response.choices[0].message
        print(tool_calls_message)
        tool_calls_response = handle_tool_call(tool_calls_message)
        messages.append(tool_calls_message)
        messages.append(tool_calls_response)
        response = openai.chat.completions.create(model=gpt_model, messages=messages)
    
    return response.choices[0].message.content

In [10]:
def handle_tool_call(tool_calls_message):
    tool_call = tool_calls_message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IXltLKk9DWRbSUe70huj9ezS', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')], annotations=[])
Tool get_ticket_price called for London
